In [1]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

import numpy as np
import math
import pandas as pd
import itertools

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 30)
#pd.options.display.float_format = '{:,}'.format 





In [2]:
def enum_category(label):
    num=0
    if label == 'Poor' : num =1
    elif label == 'Fair': num = 2
    elif label == 'Average': num = 3
    elif label == 'Good': num = 4
    elif label == 'Very Good': num = 5
    else: return label
    
    return int(num)

def log_and_normalize (data, log, norm_type):
    if log == 'log':
        data = np.log(data) 
    if norm_type == 1:
        data = (data-np.mean(data))/np.std(data)  # std normalisation
    elif norm_type == 2:
        data = (data-min(data))/(max(data)-min(data))      #  min_max_min
    elif norm_type == 3:
        data = (data-np.mean(data))/(max(data)-min(data))  # mean norm
    
    return data    

def hot_encode (data, directions):
    if directions == 'yes':
        data = pd.get_dummies(data, prefix=data.name[:3], drop_first=True)
    return data

In [3]:
df=pd.read_csv('data/kc_house_data.csv', index_col=0)

# initial cleaning operations (dropping columns, splitting , etc)
#df.drop(columns=["waterfront",'view'],inplace=True)
#df.drop(columns=["waterfront"],inplace=True)

df["grade"]=df["grade"].map(lambda x: x.split()[0]).astype(int)
df['condition'] = df['condition'].apply(enum_category)


#df[df['waterfront']=='YES']



In [4]:
# Prepairing data
#df.drop(df.index[df["bedrooms"]==33], inplace=True)

df['renovated']=np.where((df['yr_renovated'].isna() |  df['yr_renovated'] == 0), 0, 1)
df['yr_renovated'].fillna(value=df['yr_built'],inplace=True)
df['yr_renovated'] = np.where(df['yr_renovated'] == 0, df['yr_built'], df['yr_renovated'])

df['waterfront'].fillna(value='NO',inplace=True)




#df['sqft_basement'].replace('?', '0', inplace=True)
df=df[df['sqft_basement'] != '?']
df['sqft_basement']=df['sqft_basement'].astype(float).astype(int)
df['basement']= np.where(df['sqft_basement'] < 30, 0, 1)


df['sqft_basement'] = np.where((df['sqft_basement'] < 100) & (df['sqft_basement'] > -1), 1, df['sqft_basement'])
df['sqft_basement']=np.where((df['sqft_basement']==-1) & (df['sqft_living']-df['sqft_above']<=400), 
                             1, df['sqft_basement'])
df['sqft_basement']=np.where((df['sqft_basement']==-1) & (df['sqft_living']-df['sqft_above']>400) 
                             & (df['sqft_above']/df['sqft_living']<0.70),
                             (df['sqft_living']-df['sqft_above'])*0.75/df['floors'], df['sqft_basement'])
df['sqft_basement'] = np.where (df['sqft_basement'] == -1, 1, df['sqft_basement'])



#NEW FEATURES

df["age"]=2016-df['yr_built']
df["years_after_renovation"]=2016-df['yr_renovated']
df['month_of_sale']=df['date'].apply(lambda x: str.split(x,'/')[0]).astype(int)

df['after_1980']= np.where(df['yr_built'] >= 1980, 1, 0)
df['before_1940']= np.where(df['yr_built'] <= 1940, 1, 0)




df['lat_range']=pd.cut(df['lat'], 30)
df['long_range']=pd.cut(df['long'], 30)
df['coord_range']=np.array(zip(df['lat_range'],df['long_range']))
df['decade']=pd.cut(df['yr_built'], 12)

#df['coord_range'].nunique()
#FILTER

#df=df[df['yr_built']>1980]

In [5]:
#num=df.select_dtypes(include=[np.number]).drop("price", axis=1)
price=df[["price"]]

other=[]
cont=["sqft_living", 'sqft_lot', 'sqft_living15']
cat=[ "grade", 'zipcode', 'yr_built','waterfront','view']


#cont=["sqft_living"]
#cat=[ 'grade', 'zipcode', 'view', 'waterfront','yr_built']


data_fin=pd.DataFrame([])


asis=df[other]



In [6]:
# Log transform and normalize
for col in cont:
    data_fin[col]=log_and_normalize(df[col], 'log', 0)

data_fin["sqft_living"]=log_and_normalize(df["sqft_living"], 'nolog', 0)
data_fin["sqft_lot"]=log_and_normalize(df["sqft_lot"], 'log', 0)
data_fin["condition"]=log_and_normalize(df["condition"], 'log', 0)
price=log_and_normalize(price, 'log', 0)

In [7]:
# One hot encode categoricals
for col in cat:
    new_cols=hot_encode(df[col], 'yes')
    data_fin = pd.concat([data_fin, new_cols], axis=1)


In [8]:
data_fin = pd.concat([price, data_fin, asis], axis=1)

#pd.plotting.scatter_matrix(data_fin,figsize  = [14, 14]);


In [9]:
#Create OLS MODEL

pred=data_fin.drop(columns=["price"]).copy()

pred_int = sm.add_constant(pred)
model = sm.OLS(price,pred_int).fit()
model.summary()





<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.884
Model:                            OLS   Adj. R-squared:                  0.883
Method:                 Least Squares   F-statistic:                     783.9
Date:                Sat, 26 Mar 2022   Prob (F-statistic):               0.00
Time:                        21:33:50   Log-Likelihood:                 6303.7
No. Observations:               21143   AIC:                        -1.220e+04
Df Residuals:                   20939   BIC:                        -1.058e+04
Df Model:                         203                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            10.3001      0.190     54.087      0.000       9.927      10.673
sqft_living       0.0002   2.52e-06     70.568      0.000       0.000       0.000
sqft_lot          0.0715      0.002     32.568      0.000       0.067       0.076
sqft_living15     0.1659      0.007     24.765      0.000       0.153       0.179
condition         0.2080      0.008     26.719      0.000       0.193       0.223
gra_4            -0.4855      0.185     -2.623      0.009      -0.848      -0.123
gra_5            -0.4036      0.182     -2.215      0.027      -0.761      -0.046
gra_6            -0.2663      0.182     -1.465      0.143      -0.623       0.090
gra_7            -0.1289      0.182     -0.709      0.478      -0.485       0.228
gra_8            -0.0265      0.182     -0.145      0.884      -0.383       0.330
gra_9             0.0720      0.182      0.395      0.692      -0.285       0.429
gra_10            0.1388      0.182      0.762      0.446      -0.218       0.496
gra_11            0.1945      0.182      1.066      0.286      -0.163       0.552
gra_12            0.2205      0.183      1.203      0.229      -0.139       0.580
gra_13            0.3066      0.189      1.619      0.106      -0.065       0.678
zip_98002         0.0070      0.016      0.429      0.668      -0.025       0.039
zip_98003         0.0321      0.015      2.197      0.028       0.003       0.061
zip_98004         1.1226      0.014     77.875      0.000       1.094       1.151
zip_98005         0.7529      0.017     43.572      0.000       0.719       0.787
zip_98006         0.6456      0.013     49.728      0.000       0.620       0.671
zip_98007         0.6807      0.018     37.120      0.000       0.645       0.717
zip_98008         0.6868      0.015     46.461      0.000       0.658       0.716
zip_98010         0.2211      0.021     10.616      0.000       0.180       0.262
zip_98011         0.4573      0.016     28.057      0.000       0.425       0.489
zip_98014         0.2862      0.019     14.879      0.000       0.248       0.324
zip_98019         0.3061      0.016     18.625      0.000       0.274       0.338
zip_98022         0.0318      0.016      2.039      0.042       0.001       0.062
zip_98023        -0.0090      0.013     -0.709      0.478      -0.034       0.016
zip_98024         0.3928      0.023     17.284      0.000       0.348       0.437
zip_98027         0.5033      0.013     37.776      0.000       0.477       0.529
zip_98028         0.4186      0.015     28.669      0.000       0.390       0.447
zip_98029         0.6176      0.014     43.020      0.000       0.589       0.646
zip_98030         0.0527      0.015      3.511      0.000       0.023       0.082
zip_98031         0.0883      0.015      5.975      0.000       0.059       0.117
zip_98032        -0.0099      0.019     -0.522   

In [ ]:
sales_pred = model.predict()
sales_pred
plt.figure(figsize=(12, 6))
plt.plot(df['sqft_living'], df['price'], 'o')  
#plt.plot(pred_int['sqft_living'], sales_pred, 'r', linewidth=2)
plt.xlabel('Sq footage')
plt.ylabel('Price')
plt.show()

In [ ]:
#results=[]
#combs=data_fin.drop("price", axis=1).columns
##combs=num
#for comb in itertools.combinations(combs, 1):
#    test=data_fin[list(comb)].copy()
#    prep = sm.add_constant(test[list(comb)])
#    model = sm.OLS(price, prep).fit()
#    results.append([comb, model.rsquared_adj])
#
#pd.DataFrame(results).sort_values(1, ascending=False)



In [2872]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(pred,price)
linreg.score(pred, price)

0.8692358271224985

In [2873]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pred_train, pred_test, price_train, price_test = train_test_split(pred, price, test_size=0.2)

In [2874]:
linreg = LinearRegression()
linreg.fit(pred_train, price_train)
linreg.score(pred_train, price_train)

0.8690097608508968

In [2875]:
y_train = linreg.predict(pred_train)
y_test = linreg.predict(pred_test)

In [2876]:
train_residuals = y_train - price_train
test_residuals = y_test - price_test

In [2877]:
mse_train = np.sum((price_train-y_train)**2)/len(price_train)
mse_test = np.sum((price_test-y_test)**2)/len(price_test)
print('Train Mean Squarred Error:', mse_train)
print('Test Mean Squarred Error:', mse_test)

Train Mean Squarred Error: 0.0362278303625377
Test Mean Squarred Error: 0.03671439455406688


In [2878]:
mse_train

0.0362278303625377

In [2879]:
train_mse = mean_squared_error(price_train, y_train)
test_mse = mean_squared_error(price_test, y_test)
print('Train Mean Squarred Error:', train_mse)
print('Test Mean Squarred Error:', test_mse)

Train Mean Squarred Error: 0.0362278303625377
Test Mean Squarred Error: 0.03671439455406688
